In [226]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Set display options to show all columns
pd.set_option('display.max_columns', None)


# Verison 1.0

In [295]:
df = pd.read_csv("C:/Users/GTR/Documents/GitHub/Sociolla-product/data/products_all_brands.csv")

# drop rows with missing values
df.dropna(subset=['active_date'], inplace=True)
df.dropna(subset=['average_rating_by_types'], inplace=True)
df.dropna(subset=['categories'], inplace=True)

# drop unnecessary columns
df.drop(['price_by_combinations','url'], axis=1, inplace=True)
# drop duplicates
df.drop_duplicates(inplace=True)

# calculating price range mean and convert it to float
df["price_range_mean"] = df["price_range"].apply(lambda x: x.replace('Rp', ''))
df["price_range_mean"] = df["price_range_mean"].apply(lambda x: x.replace('.', ''))
df["price_range_mean"] = df["price_range_mean"].apply(lambda x: x.replace(' ', ''))
df["price_range_mean"] = df["price_range_mean"].apply(lambda x: sum([int(i) for i in x.split('-')])/len( x.split('-')))

In [296]:
# spliting categories into 3 columns
df['categories1'] = df['categories'].apply(lambda x: x.split(';')[0])
df['categories2'] = df['categories'].apply(lambda x: x.split(';')[1] if len(x.split(';')) > 1 else np.nan)
df['categories3'] = df['categories'].apply(lambda x: x.split(';')[2] if len(x.split(';')) > 2 else np.nan)


In [297]:
### adding missing value in rating_types_str from average_rating_by_types
a = df.loc[df['rating_types_str'].isna()]
a['rating_types_str'] = a['average_rating_by_types'].apply(lambda x: x.replace('.', ''))
a['rating_types_str'] = a['rating_types_str'].apply(lambda x: x.replace(':', ''))
a['rating_types_str'] = a['rating_types_str'].apply(lambda x: x.replace('"', ''))
a['rating_types_str'] = a['rating_types_str'].apply(lambda x: x.replace(' ', ''))
a['rating_types_str'] = a['rating_types_str'].str.replace(r'\d+', '')
df.loc[df['rating_types_str'].isna()] = a
df.reset_index(drop=True, inplace=True)

C:\Users\GTR\AppData\Local\Temp\ipykernel_24340\3341034531.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['rating_types_str'] = a['average_rating_by_types'].apply(lambda x: x.replace('.', ''))
C:\Users\GTR\AppData\Local\Temp\ipykernel_24340\3341034531.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['rating_types_str'] = a['rating_types_str'].apply(lambda x: x.replace(':', ''))
C:\Users\GTR\AppData\Local\Temp\ipykernel_24340\3341034531.py:5: SettingWithCopyWarning: 
A value is trying to be set o

In [299]:
# getting all types from average_rating_by_types
types = set()
gettinglist = df['average_rating_by_types'].apply(lambda x: x.replace('"', ''))
gettinglist = gettinglist.apply(lambda x: x.replace(' ', ''))
for i in gettinglist:
    for j in i.split(';'):
        types.add(j.split(':')[0])


In [300]:
# creating new columns for each types
for column_name in types:
    df[column_name] = 0

In [304]:
# filling the new columns with the value from average_rating_by_types
for i in range(0,len(df)):
    input_string = df["average_rating_by_types"][i]
    input_string = input_string.replace('"', '')
    # Split the string into key-value pairs
    pairs = input_string.split(';')
    # Create a dictionary from the key-value pairs
    for pair in pairs:
        column_name = str(pair.strip().split(':')[0])
        value = float(pair.strip().split(':')[1])
        df[column_name][i] = value

In [305]:
df.isna().sum()

brand_name                       0
product_name                     0
product_id                       0
beauty_point_earned              0
price_range                      0
active_date                      0
default_category                 0
categories                       0
rating_types_str                 0
average_rating                   0
total_reviews                    0
average_rating_by_types          0
total_recommended_count          0
total_repurchase_maybe_count     0
total_repurchase_no_count        0
total_repurchase_yes_count       0
total_in_wishlist                0
price_range_mean                 0
categories1                      0
categories2                      3
categories3                     73
star_effectiveness               0
star_durability                  0
star_value_for_money             0
star_eficiency                   0
star_texture                     0
star_scent                       0
star_long_wear                   0
star_packaging      

In [306]:
df

,brand_name,product_name,product_id,beauty_point_earned,price_range,active_date,default_category,categories,rating_types_str,average_rating,total_reviews,average_rating_by_types,total_recommended_count,total_repurchase_maybe_count,total_repurchase_no_count,total_repurchase_yes_count,total_in_wishlist,price_range_mean,categories1,categories2,categories3,star_effectiveness,star_durability,star_value_for_money,star_eficiency,star_texture,star_scent,star_long_wear,star_packaging,star_pigmentation
0,796_3ce,MULTI EYE COLOR PALETTE,97802,130,Rp 555.000 - Rp 687.000,2022-10-03T03:30:06.681Z,Eyeshadow,Makeup; Eyes; Eyeshadow,is_star_long_wear;is_star_packaging;is_star_pi...,4.920000,5,"""star_long_wear"": 5; ""star_packaging"": 4.8; ""s...",5,0,0,5,717,621000.0,Makeup,Eyes,Eyeshadow,0.000000,0.0,5.000000,0.0,5.000000,0.0,5.000000,4.800000,4.800000
1,796_3ce,VELVET LIP TINT,97810,50,Rp 264.000,2022-10-03T03:02:40.340Z,Lip Cream,Makeup; Lips; Lip Cream,is_star_long_wear;is_star_packaging;is_star_pi...,4.576190,42,"""star_long_wear"": 4.309523809523809; ""star_pac...",42,10,2,30,682,264000.0,Makeup,Lips,Lip Cream,0.000000,0.0,4.333333,0.0,4.761905,0.0,4.309524,4.785714,4.690476
2,796_3ce,MINI MULTI EYE COLOR PALETTE,97833,80,Rp 423.000,2022-10-03T03:27:01.334Z,Eyeshadow,Makeup; Eyes; Eyeshadow,is_star_long_wear;is_star_packaging;is_star_pi...,4.883333,6,"""star_long_wear"": 4.916666666666667; ""star_pac...",12,1,0,11,257,423000.0,Makeup,Eyes,Eyeshadow,0.000000,0.0,4.750000,0.0,4.916667,0.0,4.916667,5.000000,4.833333
3,796_3ce,FACE BLUSH,97801,60,Rp 300.000,2022-10-03T03:22:26.610Z,Blush,Makeup; Face; Blush,is_star_long_wear;is_star_packaging;is_star_pi...,4.858824,13,"""star_long_wear"": 4.9411764705882355; ""star_pa...",17,2,0,15,387,300000.0,Makeup,Face,Blush,0.000000,0.0,4.764706,0.0,4.882353,0.0,4.941176,5.000000,4.705882
4,796_3ce,BLUR WATER TINT,97817,50,Rp 264.000,2022-09-30T17:10:51.736Z,Lip Tint,Makeup; Lips; Lip Tint,is_star_long_wear;is_star_packaging;is_star_pi...,4.729630,51,"""star_long_wear"": 4.703703703703703; ""star_pac...",53,3,3,48,1050,264000.0,Makeup,Lips,Lip Tint,0.000000,0.0,4.629630,0.0,4.777778,0.0,4.703704,4.777778,4.759259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4080,15371_youvit,Youvit Acno 7 days,103726,0,Rp 29.900,2023-06-06T02:47:32.800Z,Beauty Suplements,Supplement; Beauty Suplements,is_star_effectiveness;is_star_eficiency;is_sta...,4.333333,6,"""star_effectiveness"": 4.166666666666667; ""star...",4,3,0,3,25,29900.0,Supplement,Beauty Suplements,NaN,4.166667,0.0,4.333333,4.5,0.000000,0.0,0.000000,0.000000,0.000000
4081,273_yves-rocher,Shower Gel,63865,30,Rp 119.000 - Rp 179.000,2023-09-28T09:37:13.539Z,Body Wash,Bath & Body; Body Cleanser; Body Wash,is_star_effectiveness;is_star_packaging;is_sta...,4.660714,14,"""star_effectiveness"": 4.857142857142857; ""star...",14,4,0,10,332,149000.0,Bath & Body,Body Cleanser,Body Wash,4.857143,0.0,4.571429,0.0,4.785714,0.0,0.000000,4.428571,0.000000
4082,273_yves-rocher,Relaxing Shower Gel Olive Petitgrain,68293,20,Rp 119.000,2023-04-13T07:35:43.813Z,Body Wash,Chăm sóc cơ thể; Làm sạch; Sữa tắm,is_star_effectiveness;is_star_packaging;is_sta...,4.607143,7,"""star_effectiveness"": 4.571428571428571; ""star...",7,1,1,5,171,119000.0,Chăm sóc cơ thể,Làm sạch,Sữa tắm,4.571429,0.0,4.285714,0.0,4.714286,0.0,0.000000,4.857143,0.000000
4083,273_yves-rocher,Repair Hair Mask,36357,50,Rp 289.000,2022-10-27T07:33:19.380Z,Hair Mask,Hair Care; Hair Treatment; Hair Mask,is_star_effectiveness;is_star_packaging;is_sta...,4.750000,5,"""star_effectiveness"": 5; ""star_packaging"": 5; ...",5,1,0,4,58,289000.0,Hair Care,Hair Treatment,Hair Mask,5.000000,0.0,4.000000,0.0,5.000000,0.0,0.000000,5.000000,0.000000
